<a href="https://colab.research.google.com/github/alexandergribenchenko/Data_Science_Toolkit/blob/main/Pyspark/DS_PySpark_01_Create_Dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. Instalación y carga de Pyspark

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 54.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e9c373fcc08a6dc4647facc9a62c9deccf2cba0012a1a5bb919695d8650d259c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

# Librerías necesarias

In [ ]:
from pyspark.sql.types import StringType, BooleanType, FloatType, IntegerType, DoubleType, DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import sum, col, desc, asc, count, countDistinct, round, max, min, avg
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.window import Window

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml import Model
from pyspark.ml import Estimator
from pyspark import SparkFiles

from datetime import datetime
import numpy as np

# 01. Create Dataframe from: csv file (Drive, Databricks-Datalake)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
path_drive = '/content/drive/MyDrive/Colab Notebooks/train.csv'
path_databricks_datalake = '/mnt/'+'adv/Network_Planning/Caso_02/Sample_Input_Data/Input_03_Avianca_RT.txt'

In [ ]:
df = spark.read.load(path_drive,format="csv", sep=",", inferSchema="true", header="true")
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

# 02. Create Dataframe from: csv file (Github)

In [ ]:
path_github = 'https://raw.githubusercontent.com/alexandergribenchenko/Data_Science_Datasets/main/Kaggle_Titanic/train.csv'
file_name = path_github.rsplit('/', 1)[-1]
file_name

'train.csv'

In [ ]:
spark.sparkContext.addFile(path_github)

In [ ]:
df = spark.read.load(SparkFiles.get(file_name),format="csv", sep=",", inferSchema="false", header="true")
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [ ]:
df.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



# 03. Create Dataframe from: tuples

In [ ]:
data = [("James", "Sales", 3000), \
    ("Michael", "Sales", 4600), \
    ("Robert", "Sales", 4100), \
    ("Maria", "Finance", 3000), \
    ("James", "Sales", 3000), \
    ("Scott", "Finance", 3300), \
    ("Jen", "Finance", 3900), \
    ("Jeff", "Marketing", 3000), \
    ("Perro", "Sales", 15), \
    ("Perro", "Sales", 15) \
  ]
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = data, schema = columns)

In [ ]:
df.show()

# 04. Create Dataframe from: pandas dataframe

In [ ]:
import pandas as pd

In [ ]:
dict_input = { 'col_01': [1,2,3,4], 
               'col_02': ['gato','perro','gato','perro'],
               'col_03': [1,5,6,7]
             }

df_pandas = pd.DataFrame.from_dict(dict_input)
df_pandas

,col_01,col_02,col_03
0,1,gato,1
1,2,perro,5
2,3,gato,6
3,4,perro,7


In [ ]:
df_spark = spark.createDataFrame(df_pandas)
df_spark.show()

+------+------+------+
|col_01|col_02|col_03|
+------+------+------+
|     1|  gato|     1|
|     2| perro|     5|
|     3|  gato|     6|
|     4| perro|     7|
+------+------+------+



# 05. Create Dataframe from: Azure Data Base

In [ ]:
jdbc_Hostname = "replace_host_name.database.windows.net"
jdbc_Database = "replace_database_name"
jdbc_Port = "XXXX"
jdbc_Properties = {"user" : "replace_user_name", "password" : "replace_password" }

In [ ]:
jdbc_url = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbc_Hostname,jdbc_Port,jdbc_Database)

In [ ]:
df = spark.read.jdbc(url=jdbc_url, table="xxx.replace_table_name", properties=jdbc_Properties) 

# 06. Save Dataframe to: csv file (Drive, Github, Databricks-Datalake)

In [ ]:
path_drive_output = '/content/drive/MyDrive/Colab Notebooks/df_spark_output.csv'
path_databricks_datalake = '/mnt/'+'adv/Network_Planning/Caso_02/Sample_Input_Data/df_spark_output.csv'

In [ ]:
df_spark.write.option("sep","|").option("header",True).mode('overwrite').csv(path_drive_output)